# Treasury Trading Optimization: R Comparison
This notebook processes 2024 U.S. Treasury real yield curve data and solves a simplified optimization problem using `lpSolve`, as not all engines (e.g., MOSEK, Gurobi) have direct R interfaces. It replicates the Python notebook's plots.

In [ ]:
install.packages(c("tidyverse", "httr", "lpSolve", "ggplot2"))
library(tidyverse)
library(httr)
library(lpSolve)
library(ggplot2)

## Data Retrieval

In [ ]:
# Fetch 2024 Treasury data
url <- "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/2024/all?type=daily_treasury_real_yield_curve&field_tdr_date_value=2024&page&_format=csv"
response <- GET(url)
writeBin(content(response, "raw"), "treasury_2024.csv")

# Load and process
data <- read_csv("treasury_2024.csv")
rates <- data %>% 
  select(Date, `5 Yr`, `10 Yr`, `20 Yr`, `30 Yr`) %>% 
  drop_na() %>% 
  mutate(Date = as.Date(Date))
print(tail(rates))

## Problem Definition
- **Objective**: Minimize $\sum (w_i \cdot r_i + f_i)$
- **Constraints**:
  - $\sum w_i = 1$
  - $w_i \geq 0.02$ (haircut)
  - $w_i \geq 0$
- **Parameters**:
  - $r_i$: Latest yields
  - $h_i = 0.02$
  - $f_i = 0.001$

In [ ]:
# Latest rates
r <- as.numeric(tail(rates, 1)[, 2:5]) / 100
h <- rep(0.02, 4)
f <- rep(0.001, 4)
n <- length(r)

# Objective: r + f
obj <- r + f

# Constraints: sum(w) = 1, w >= h
con <- rbind(t(rep(1, n)), diag(n))
dir <- c("=", rep(">=", n))
rhs <- c(1, h)

# Solve
sol <- lp("min", obj, con, dir, rhs)
w_lpsolve <- sol$solution
cost_lpsolve <- sol$objval
cat("lpSolve: Weights =", w_lpsolve, ", Cost =", cost_lpsolve, "\n")

## Visualization

In [ ]:
# Mock results (only lpSolve run in R)
results <- data.frame(
  Engine = c("lpSolve"),
  Cost = c(cost_lpsolve)
)

ggplot(results, aes(x = Engine, y = Cost)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  theme_minimal() +
  labs(title = "Optimization Engine Cost Comparison (Feb 19, 2025)", y = "Total Funding Cost") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

# Weights heatmap
weights <- data.frame(
  Engine = "lpSolve",
  t(w_lpsolve)
)
colnames(weights) <- c("Engine", "5 Yr", "10 Yr", "20 Yr", "30 Yr")
weights_long <- pivot_longer(weights, -Engine, names_to = "Maturity", values_to = "Weight")

ggplot(weights_long, aes(x = Engine, y = Maturity, fill = Weight)) +
  geom_tile() +
  geom_text(aes(label = round(Weight, 3))) +
  scale_fill_gradient(low = "yellow", high = "blue") +
  labs(title = "Allocation Weights by Engine") +
  theme_minimal()

## Notes
- **Solvers**: Only `lpSolve` used due to limited R support for MOSEK, Gurobi, etc. Export Python results to CSV for full comparison.
- **Data**: Adjust URL parsing if CSV format changes.
- **Extensions**: Use `Rglpk` or `ROI` for alternative R solvers.